# Parte 1: Utilitidade

## Importar os pacotes necessários

In [5]:
# importar os pacotes necessários para fazer análise
import pandas as pd, numpy as np
import os, re
import requests
import zipfile

## Definir funções para processar os dados

In [6]:
# definir função que baixa os dados
def baixar_tse():
    URL = 'http://agencia.tse.jus.br/estatistica/sead/odsele/motivo_cassacao/motivo_cassacao_2016.zip'
    r = requests.get(URL)
    with open('../dados/motivo_cassacao2016.zip', 'wb') as arquivo:
        arquivo.write(r.content)
    return 'Motivo(s) para Indeferimento 2016 baixado(s) com sucesso.'

# definir função de descomprime arquivo do tse
def unzip_tse():
    with zipfile.ZipFile('../dados/motivo_cassacao2016.zip', 'r') as zip:
        zip.extractall('../dados')
    return 'Arquivo(s) descomprimido(s) com sucesso.'

# definir função para juntar os dados do tse
def juntar_tse():

    regex = re.compile(r'motivo.*csv$')
    juntar_tse.kwargs = {
        'engine': 'python', 'encoding': 'latin1', 'sep':';', 'quoting': 1,
        'dtype': str
    }

    arquivos = os.listdir('../dados')
    arquivos = list(filter(regex.search, arquivos))
    arquivos = [
        os.path.join(os.path.realpath('../dados'), arquivo) \
        for arquivo in arquivos
    ]

    dados = pd.concat(
        [pd.read_csv(arquivo, **juntar_tse.kwargs) for arquivo in arquivos],
        ignore_index=True
    )

    dados = dados[dados['DS_MOTIVO_CASSACAO'].notna()]
    # dados.drop(dados.columns[-1], axis=1, inplace=True)

    print('Arquivo(s) juntado(s) com sucesso.')
    return dados

## Executar funções

In [7]:
# executar função
baixar_tse()

'Motivo(s) para Indeferimento 2016 baixado(s) com sucesso.'

In [8]:
# executar função
unzip_tse()

'Arquivo(s) descomprimido(s) com sucesso.'

In [16]:
# executar função
candidaturas = juntar_tse()

Arquivo(s) juntado(s) com sucesso.


## Exibir dados preliminares

In [17]:
# mostrar os dados
candidaturas.sample(10)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,SQ_CANDIDATO,DS_MOTIVO_CASSACAO
412,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,AL,28134,OLHO D'ÁGUA DAS FLORES,20000001534,Ausência de requisito de registro
2390,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,BA,35130,EUCLIDES DA CUNHA,50000010638,Ausência de requisito de registro
13046,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,SP,70270,SANTA CRUZ DAS PALMEIRAS,250000072958,Ausência de requisito de registro
9870,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,PA,04251,BARCARENA,140000020970,Ausência de requisito de registro
17571,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,MG,54194,VAZANTE,130000078672,Ausência de requisito de registro
9977,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,RS,87734,OSÓRIO,210000009300,Ficha limpa (LC 64/90)
5508,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,PR,76295,JACAREZINHO,160000015311,Ausência de requisito de registro
13077,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,SP,64750,GUARUJÁ,250000069128,Ausência de requisito de registro
18073,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,MT,90204,GLÓRIA D'OESTE,110000001495,Ausência de requisito de registro
15602,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,MG,41238,BELO HORIZONTE,130000084207,Ausência de requisito de registro


In [18]:
# verificar os motivos para cassação
list(candidaturas['DS_MOTIVO_CASSACAO'].unique())

['Ausência de requisito de registro ',
 'Indeferimento de partido ou coligação.',
 'Compra de voto (Lei 9.504/97).',
 'Ficha limpa (LC 64/90)',
 'Abuso de poder (LC 64/90)',
 'Gasto ilícito de recursos (Lei 9.504/97).',
 'Impugnação',
 'Conduta vedada (Lei 9.504/97).']

## Focar no Paraná

In [21]:
# focar em 10 candidatos no paraná
candidaturas_pr = candidaturas[candidaturas['SG_UF'] == 'PR']
amostra = candidaturas_pr.sample(10, random_state=67)
amostra.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'SG_UF', 'SG_UE',
       'NM_UE', 'SQ_CANDIDATO', 'DS_MOTIVO_CASSACAO'],
      dtype='object')

In [22]:
# vamos achar as documentos processuais?
candidatos_pr = pd.read_csv(
    '../dados/consulta_cand_2016_PR.csv', **juntar_tse.kwargs
)

# puxar os números do protocolo da candidatura
amostra = amostra.merge(candidatos_pr, on='SQ_CANDIDATO')
amostra['NR_PROTOCOLO_CANDIDATURA']

0    1194392016
1    1225872016
2    1013362016
3    1134682016
4    1065522016
5    1182182016
6    1247862016
7    1969862016
8    1211272016
9    1151382016
Name: NR_PROTOCOLO_CANDIDATURA, dtype: object

In [25]:
# criar link para raspar documentos
documentos_link = [
    f'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?' \
    f'nprot={nprot}&comboTribunal=pr' \
    for nprot in amostra['NR_PROTOCOLO_CANDIDATURA'].to_list()
]
print(documentos_link)

['http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1194392016&comboTribunal=pr',
 'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1225872016&comboTribunal=pr',
 'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1013362016&comboTribunal=pr',
 'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1134682016&comboTribunal=pr',
 'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1065522016&comboTribunal=pr',
 'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1182182016&comboTribunal=pr',
 'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1247862016&comboTribunal=pr',
 'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1969862016&comboTribunal=pr',
 'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1211272016&comboTribunal=pr',
 'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1151382016&comboTribunal=pr']

## Baixar e processar os dados

In [ ]:
# # baixar os arquivos
# raspador = tse.scraper(navegador_automatico.browser)

# # baixar todos
# for i, url in enumerate(documentos_link):
#     raspador.decision(url=url, filename=f'decisão_{i+1}')

# # fechar
# navegador_automatico.browser.quit()

In [27]:
# construir path para os arquivos
decisões = [f'../dados/decisão_{i}.html' for i in range(0, 10)]

# transformar em tabela
sumário = parser(decisões[1])
sumário.parse_summary()
pd.DataFrame(sumário.parse_summary()).T

,0
case,Nº 0000299-72.2016.6.16.0127 - REGISTRO DE CAN...
town,NOVA OLÍMPIA-PRN.° Origem:
prot,1225872016 - 15/08/2016 18:13
claimants,COLIGAÇÃO GOVERNANDO COM O POVO (PMDB / PSC / ...
defendant,"VERA ILSA BALDUINO DA SILVA, CARGO VEREADOR, N..."
judge,FERNANDA BATISTA DORNELLES
subject,DIREITO ELEITORAL - Eleições - Candidatos - Re...
district,127ZE-127 ZONA ELEITORAL
stage,09/02/2017 12:21-Arquivado na seção


In [29]:
# criar banco de dados
sumários = pd.concat(
    [pd.DataFrame(parser(decisão).parse_summary()) for decisão in decisões],
    ignore_index=True
)
print(sumários)

                                                case  \
0  Nº 0000304-61.2016.6.16.0041 - REGISTRO DE CAN...   
1  Nº 0000299-72.2016.6.16.0127 - REGISTRO DE CAN...   
2  Nº 0000050-36.2016.6.16.0123 - REGISTRO DE CAN...   
3  Nº 0000161-14.2016.6.16.0028 - REGISTRO DE CAN...   
4  Nº 0000186-20.2016.6.16.0095 - REGISTRO DE CAN...   
5  Nº 0000280-70.2016.6.16.0061 - REGISTRO DE CAN...   
6  Nº 0000180-76.2016.6.16.0074 - REGISTRO DE CAN...   
7  Nº 0000259-85.2016.6.16.0161 - REGISTRO DE CAN...   
8  Nº 0000271-55.2016.6.16.0111 - REGISTRO DE CAN...   
9  Nº 0000186-10.2016.6.16.0163 - REGISTRO DE CAN...   

                                    town                           prot  \
0                 LONDRINA-PRN.° Origem:  1194392016 - 15/08/2016 17:09   
1             NOVA OLÍMPIA-PRN.° Origem:  1225872016 - 15/08/2016 18:13   
2  SÃO JORGE DO PATROCÍNIO-PRN.° Origem:  1013362016 - 12/08/2016 18:24   
3                APUCARANA-PRN.° Origem:  1134682016 - 15/08/2016 14:58   
4       

In [30]:
# onde estão estes processos? nas seguintes zonas eleitorais
sumários['district']

0    041ZE-041 ZONA ELEITORAL
1    127ZE-127 ZONA ELEITORAL
2    123ZE-123 ZONA ELEITORAL
3    028ZE-028 ZONA ELEITORAL
4    095ZE-095 ZONA ELEITORAL
5    061ZE-061 ZONA ELEITORAL
6    074ZE-074 ZONA ELEITORAL
7    161ZE-161 ZONA ELEITORAL
8    111ZE-111 ZONA ELEITORAL
9    163ZE-163 ZONA ELEITORAL
Name: district, dtype: object

## Parte 3: Processamento de Linguagem Natural (NLP)